In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display as disp, clear_output
from functools import partial
from autoplan.token import PyretTokenizer
from scripts.rainfall_ingest import load_new_labels, ingest_dataset
from grammars.rainfall.ocaml import Labels
from pickle_cache import PickleCache
import pickle
pcache = PickleCache()

In [6]:
REPO_DIR = os.path.expanduser('~/autoplan')
DATA_DIR = f'{REPO_DIR}/data/rainfall/raw'
CODE_DIR = f'{DATA_DIR}/Fall2013-RawData'

def read_coding_csv(name):
    return pd.read_csv(f'{DATA_DIR}/Fall2013Coding{name}.csv', index_col=0, header=None).T

In [7]:
plan_codes = pd.read_csv(f'{DATA_DIR}/PlanCodes-codes.csv')

def read_and_join_coding(name):
    coding_csv = read_coding_csv(name)
    valid_entries = coding_csv[coding_csv.PlanStructure.isnull()]
    combined_entries = valid_entries.set_index('PlanStructure').join(plan_codes.set_index('Code'))
    return combined_entries

In [8]:
coding_csv = read_coding_csv('T1Acc')
missing_entries = coding_csv[coding_csv.PlanStructure.isnull()]

In [ ]:
len()

In [12]:
sources = {}
tokenizer = PyretTokenizer()
for _, row in missing_entries.iterrows():
    id = row.ID
    path = f'{CODE_DIR}/T1Acc/{id}/cs019-2013-rainfall/rainfall-program.current.arr'
    if os.path.isfile(path):
        try:
            sources[id] = tokenizer.tokenize(open(path, 'r').read())[1]
        except UnicodeDecodeError:
            pass

In [54]:
labels = {}

In [55]:
plans = ['Clean First', 'Clean in SC', 'Single Loop']

gen = iter(sources.items())

def render():
    id, source = next(gen)
    
    def on_click(plan, _):
        labels[id] = plan
        render()
        
    btns = []
    for p in plans:
        btn = widgets.Button(description=p)        
        btn.on_click(partial(on_click, p))
        btns.append(btn)
        
    clear_output()
    disp(widgets.HBox(btns))
    print(source)

render()

let rainfall (my_list : 'a list) =
  (let rec helper (alon : 'a list) (sum : int) (count : int) =
     (match (alon, sum) with
      | ([], v) -> if count > 0 then (v + 1) / count else 0
      | (hd::tl, v) ->
          if hd = (-999)
          then (if count > 0 then v / count else 0)
          else
            if hd >= 0
            then helper tl (v + hd) (count + 1)
            else helper tl v count : int) in
   helper my_list 0 0 : int)



StopIteration: 

In [70]:
bad = ['116']

In [75]:
planstr_to_plan = {
    'Clean First': Labels.CleanFirst,
    'Clean in SC': Labels.CleanInSC,
    'Single Loop': Labels.SingleLoop
}

plan_labels = {k: planstr_to_plan[v] for k, v in labels.items() if k not in bad}

In [78]:
pcache.set('ocaml_labels', plan_labels)

In [82]:
pickle.dump(plan_labels, open(f'{DATA_DIR}/T1-newlabels.pkl', 'wb'))

In [87]:
ds = ingest_dataset('T1')
new_ds = load_new_labels(ds.vocab_index)

Skipped 16 programs
